# 서울시 미용실 데이터 - NAVER Map(v4) 데이터 전처리(1)
## 작성자(최종수정일) : 서동원(2020.04.20)
- 검색 거절 데이터(rejected_data) 조건 수정하여 다시 검색하기 

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import re
import requests
import time
pd.set_option('display.max_columns',50) # DataFrame truncation 없이 보기
pd.set_option('display.max_rows',100)
from tqdm import tqdm
import math

## 네이버 지도 검색 거절 데이터

In [2]:
# 데이터 읽어오기
naver_search_rejected_raw = pd.read_csv('./naver_map_seoul_salon_output_rejected_data_2020-04-19.csv', index_col=0)
pre_rejected = len(naver_search_rejected_raw)

In [3]:
# 중복 제거
naver_search_rejected_raw.drop_duplicates(inplace=True)

print('중복 제거 전 개수 :', pre_rejected)
print('중복 제거 후 개수 :', len(naver_search_rejected_raw))
pre_rejected = len(naver_search_rejected_raw)

중복 제거 전 개수 : 3757
중복 제거 후 개수 : 3756


In [4]:
naver_search_rejected_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3756 entries, 0 to 11642
Data columns (total 7 columns):
지역        3756 non-null object
업소명       3756 non-null object
소재지도로명    3741 non-null object
검색어       3756 non-null object
검색건수      3756 non-null int64
원인        3756 non-null object
수집일       3756 non-null object
dtypes: int64(1), object(6)
memory usage: 234.8+ KB


In [5]:
# '원인' 컬럼
naver_search_rejected_raw['원인'].unique()

array(['검색결과 없음', '검색건수 100건 초과'], dtype=object)

검색하지 않은 원인은 두가지 뿐이다.
    - (1) 검색결과 없음
    - (2) 검색결과 100건 초과

In [6]:
# 검색 결과가 없는 데이터
naver_search_rejected_raw[naver_search_rejected_raw['원인'] == '검색결과 없음'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3739 entries, 0 to 11642
Data columns (total 7 columns):
지역        3739 non-null object
업소명       3739 non-null object
소재지도로명    3736 non-null object
검색어       3739 non-null object
검색건수      3739 non-null int64
원인        3739 non-null object
수집일       3739 non-null object
dtypes: int64(1), object(6)
memory usage: 233.7+ KB


In [7]:
# 검색 결과가 100건 초과되는 데이터
naver_search_rejected_raw[naver_search_rejected_raw['원인'] == '검색건수 100건 초과'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 1770 to 3655
Data columns (total 7 columns):
지역        17 non-null object
업소명       17 non-null object
소재지도로명    5 non-null object
검색어       17 non-null object
검색건수      17 non-null int64
원인        17 non-null object
수집일       17 non-null object
dtypes: int64(1), object(6)
memory usage: 1.1+ KB


In [8]:
# 100건 초과 데이터 조회
too_many_result = naver_search_rejected_raw[naver_search_rejected_raw['원인'] == '검색건수 100건 초과']
too_many_result

,지역,업소명,소재지도로명,검색어,검색건수,원인,수집일
1770,동작구,헤어꾼,"서울특별시 동작구 상도로37길 81, (상도동)",헤어꾼,0,검색건수 100건 초과,2020-04-09
1851,마포구,초이미용실,"서울특별시 마포구 홍익로 지하 10, (서교동)",초이미용실,0,검색건수 100건 초과,2020-04-09
1852,마포구,세마헤어,"서울특별시 마포구 양화로18안길 22, (동교동,외1(3층))",세마헤어,0,검색건수 100건 초과,2020-04-09
1931,마포구,홍대 살롱,"서울특별시 마포구 와우산로29나길 24-4, 1층 (서교동)",홍대 살롱,0,검색건수 100건 초과,2020-04-09
2566,송파구,양지헤어쎈스,"서울특별시 송파구 풍성로23길 40, (풍납동)",양지헤어쎈스,0,검색건수 100건 초과,2020-04-09
2990,영등포구,초원,NaN,초원,0,검색건수 100건 초과,2020-04-09
3009,영등포구,예지미용실,NaN,예지미용실,0,검색건수 100건 초과,2020-04-09
3022,영등포구,마샬,NaN,마샬,0,검색건수 100건 초과,2020-04-09
3055,영등포구,그린헤어샵,NaN,그린헤어샵,0,검색건수 100건 초과,2020-04-09
3081,영등포구,주노미용실,NaN,주노미용실,0,검색건수 100건 초과,2020-04-09


In [9]:
# 100건 이상 데이터 인덱스
too_many_result.index

Int64Index([1770, 1851, 1852, 1931, 2566, 2990, 3009, 3022, 3055, 3081, 3311,
            3314, 3540, 3552, 3574, 3575, 3655],
           dtype='int64')

In [10]:
# 검색에 사용한 데이터 읽어오기
input_data = pd.read_csv('./naver_map_seoul_salon_input_data_2020-04-09_2020-04-19.csv', index_col=0)
input_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15914 entries, 0 to 15913
Data columns (total 7 columns):
지역         15914 non-null object
업소명        15914 non-null object
소재지전화번호    10287 non-null object
행정동명       15873 non-null object
소재지도로명     15860 non-null object
소재지지번      15873 non-null object
수집일        15914 non-null object
dtypes: object(7)
memory usage: 994.6+ KB


In [11]:
# 검색건수 100건 이상 데이터 인덱스로 조회하기
input_data.iloc[too_many_result.index]

,지역,업소명,소재지전화번호,행정동명,소재지도로명,소재지지번,수집일
1770,강북구,머리사랑,NaN,미아동,"서울특별시 강북구 오패산로52길 지하 66, 1호 (미아동)",서울특별시 강북구 미아동 258번지 478호,2020-04-09
1851,강북구,아이시떼루 헤어클럽,NaN,우이동,"서울특별시 강북구 삼양로163나길 2, (우이동)",서울특별시 강북구 우이동 30번지 19호,2020-04-09
1852,강북구,채규희헤어클럽,NaN,우이동,"서울특별시 강북구 삼양로163나길 2, (우이동)",서울특별시 강북구 우이동 30번지 19호,2020-04-09
1931,강북구,헤어스케치,NaN,수유제1동,"서울특별시 강북구 삼양로116길 35, 101호 (수유동, 수유빌라트)",서울특별시 강북구 수유동 245번지 99호 수유빌라트-101,2020-04-09
2566,강서구,비비헤어,02 20612280,등촌제2동,"서울특별시 강서구 등촌로 213, (등촌동)",서울특별시 강서구 등촌동 510번지 13호,2020-04-09
2990,강서구,최가을헤어드레서,0226029416,화곡본동,"서울특별시 강서구 화곡로 194-15, 2층 (화곡동)",서울특별시 강서구 화곡동 98번지 238호 2층,2020-04-09
3009,강서구,Salonde'(살롱드) 시유,NaN,가양제1동,"서울특별시 강서구 양천로 344, 마곡센트럴대방디엠시티오피스텔 125-2호 (마곡동)",서울특별시 강서구 마곡동 776번지 2호 마곡센트럴대방디엠시티오피스텔-125-2,2020-04-09
3022,강서구,에스헤어디자인,NaN,가양제3동,"서울특별시 강서구 양천로 507, 상가동 1층 110,111호 (가양동, 강나루현대...","서울특별시 강서구 가양동 448번지 1호 강나루현대아파트 (지상 1층) 110,...",2020-04-09
3055,강서구,헤어달디,NaN,가양제1동,"서울특별시 강서구 마곡중앙로 161-1, 마곡나루역캐슬파크 211호 (마곡동)",서울특별시 강서구 마곡동 760번지 2호 마곡나루역캐슬파크-211,2020-04-09
3081,강서구,라임 헤어 살롱,NaN,가양제1동,"서울특별시 강서구 허준로 16, 성지빌딩 103호 (가양동)",서울특별시 강서구 가양동 1458번지 7호 성지빌딩-103,2020-04-09


- 검색건수 100건이상 데이터의 인덱스와 검색에 사용한 input 데이터의 인덱스가 맞지 않는다.
- 다른 방법으로 조회해보자

In [12]:
# '지역'과 '업소명' 컬럼값 붙이기 (Series 연산)
too_many_result['지역'] + too_many_result['업소명']

1770         동작구헤어꾼
1851       마포구초이미용실
1852        마포구세마헤어
1931       마포구홍대 살롱
2566      송파구양지헤어쎈스
2990         영등포구초원
3009      영등포구예지미용실
3022         영등포구마샬
3055      영등포구그린헤어샵
3081      영등포구주노미용실
3311       은평구머리하는날
3314      은평구헤어본미용실
3540           중구광희
3552         중구준미용실
3574    중구박승철헤어스투디오
3575         중구헤어클럽
3655    중랑구에스(S) 헤어
dtype: object

In [13]:
# '지역' 컬럼과 '업소명' 컬럼 값을 붙인 결과가 같은 데이터만 인덱싱
df_list = [input_data[(input_data['지역'] + input_data['업소명']) == x] for x in (too_many_result['지역'] + too_many_result['업소명'])]

In [14]:
# DataFrame concat
need_re_crawling = pd.concat(df_list)
need_re_crawling

,지역,업소명,소재지전화번호,행정동명,소재지도로명,소재지지번,수집일
7545,동작구,헤어꾼,NaN,상도제3동,"서울특별시 동작구 상도로37길 81, (상도동)",서울특별시 동작구 상도동 485번지 1호,2020-04-09
7919,마포구,초이미용실,02 337 9631,서교동,"서울특별시 마포구 홍익로 지하 10, (서교동)",서울특별시 마포구 서교동 486번지 서교푸르지오101동지하1층135호,2020-04-09
7921,마포구,세마헤어,02 333 6777,서교동,"서울특별시 마포구 양화로18안길 22, (동교동,외1(3층))",서울특별시 마포구 동교동 169번지 22호 외1(3층),2020-04-09
8219,마포구,홍대 살롱,02 63381114,대흥동,"서울특별시 마포구 백범로 98, 2,3층 (대흥동)","서울특별시 마포구 대흥동 265번지 1호 2,3층",2020-04-09
8221,마포구,홍대 살롱,02 63381114,서교동,"서울특별시 마포구 와우산로29나길 24-4, 1층 (서교동)",서울특별시 마포구 서교동 335번지 13호 1층,2020-04-09
10984,송파구,양지헤어쎈스,02 4746364,풍납1동,"서울특별시 송파구 바람드리길 42, 1층 1호 (풍납동)",서울특별시 송파구 풍납동 148번지 1호,2020-04-09
10985,송파구,양지헤어쎈스,02 4746364,풍납1동,"서울특별시 송파구 풍성로23길 40, (풍납동)",서울특별시 송파구 풍납동 151번지 2호,2020-04-09
12847,영등포구,초원,0226347416,당산제1동,"서울특별시 영등포구 당산로16길 17-1, 1층 104호 (당산동1가)",서울특별시 영등포구 당산동1가 398번지 1호 1층-104,2020-04-09
12848,영등포구,초원,0226347416,당산제1동,NaN,서울특별시 영등포구 당산동1가 186번지 51호,2020-04-09
12922,영등포구,예지미용실,0226714586,양평제1동,NaN,서울특별시 영등포구 양평동1가 28번지,2020-04-09


In [15]:
# 모듈화한 네이버 지도 검색 함수 import
from data_crawling_func.NaverMapCrawling import run_naver_map_search

In [16]:
# 재검색
tmp = run_naver_map_search(need_re_crawling, 0, len(need_re_crawling))

try row-num : 7545
검색어 : 헤어꾼 미용실
검색결과 1건
[2020-04-20 12:30:55.329798] 검색 및 저장 성공 !!
성공 : 1 | 0건 검색 : 0 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 7919
검색어 : 초이미용실 미용실
검색결과 1건
[2020-04-20 12:30:59.997534] 검색 및 저장 성공 !!
성공 : 2 | 0건 검색 : 0 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 7921
검색어 : 세마헤어 미용실
검색결과 1건
[2020-04-20 12:31:05.184596] 검색 및 저장 성공 !!
성공 : 3 | 0건 검색 : 0 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 8219
검색어 : 홍대 살롱 미용실
검색결과 30건
[2020-04-20 12:31:14.300239] 검색 및 저장 성공 !!
성공 : 4 | 0건 검색 : 0 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 8221
검색어 : 홍대 살롱 미용실
검색결과 75건
[2020-04-20 12:31:17.385575] 검색결과 50건 초과 !!
재시도 - 검색어 : "홍대 살롱" 미용실
검색결과 50건
[2020-04-20 12:31:32.109131] 검색 및 저장 성공 !!
성공 : 5 | 0건 검색 : 0 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 10984
검색어 : 양지헤어쎈스 미용실
검색결과 0건
[2020-04-20 12:31:34.907875] 검색결과 없음 !!
성공 : 5 | 0건 검색 : 1 | 50건이상 검색 : 0 | 캡챠 우회 : 0

try row-num : 10985
검색어 : 양지헤어쎈스 미용실
검색결과 0건
[2020-04-20 12:31:37.936860] 검색결과 없음 !!
성공 : 5 | 0건 검색 : 2 | 50건이상 검색 : 0 | 캡챠 우회 

In [17]:
# 원래 검색 완료한 데이터 읽어오기
naver_search_all = pd.read_csv('./naver_map_seoul_salon_output_data_2020-04-19.csv', index_col=0)
naver_search_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26796 entries, 0 to 26795
Data columns (total 9 columns):
업소ID       26796 non-null object
업소명        26794 non-null object
소재지전화번호    19231 non-null object
소재지도로명     26796 non-null object
카테고리       26796 non-null object
검색인덱스      26796 non-null int64
검색어        26796 non-null object
검색건수       26796 non-null int64
수집일        26796 non-null object
dtypes: int64(2), object(7)
memory usage: 2.0+ MB


In [19]:
# 방금 재검색 완료한 데이터 읽어오기
naver_re_search_all = tmp[0]
naver_re_search_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 10 columns):
업소ID       106 non-null object
업소명        106 non-null object
소재지전화번호    93 non-null object
소재지도로명     106 non-null object
카테고리       106 non-null object
검색인덱스      106 non-null object
검색주소타입     106 non-null object
검색어        106 non-null object
검색건수       106 non-null object
수집일        106 non-null object
dtypes: object(10)
memory usage: 8.4+ KB


In [20]:
# 두 DF concat해서 csv로 저장하기
pd.concat([naver_search_all, naver_re_search_all], sort=False).to_csv(
      './data/naver_map_seoul_salon_output_data_2020-04-20_modified.csv'
    , sep=','
    , encoding='utf-8'
)

In [21]:
# 재검색 이후 다시 거절된 데이터
naver_re_rejected = tmp[1]
naver_re_rejected

,지역,업소명,검색주소,검색주소타입,검색어,검색건수,원인,수집일
10984,송파구,양지헤어쎈스,"서울특별시 송파구 바람드리길 42, 1층 1호 (풍납동)",1,양지헤어쎈스 미용실,0,검색결과 없음,2020-04-09
10985,송파구,양지헤어쎈스,"서울특별시 송파구 풍성로23길 40, (풍납동)",1,양지헤어쎈스 미용실,0,검색결과 없음,2020-04-09
12958,영등포구,마샬,"서울특별시 영등포구 대림로31길 38, 신한승빌딩 2층 202호 (대림동)",1,마샬 미용실,0,검색결과 없음,2020-04-09
12959,영등포구,마샬,서울특별시 영등포구 대림동 729번지 17호,2,마샬 미용실,0,검색결과 없음,2020-04-09
14198,은평구,머리하는날,서울특별시 은평구 수색동 1번지 대림한솔타운아파트 210호,2,머리하는날 미용실,0,검색결과 없음,2020-04-09
14204,은평구,헤어본미용실,서울특별시 은평구 수색동 산 1번지,2,헤어본미용실 미용실,0,검색결과 없음,2020-04-09
15282,중구,박승철헤어스투디오,서울특별시 중구 황학동 2198번지 E마트황학점1층 미용실코너,2,박승철헤어스투디오 미용실,0,검색결과 없음,2020-04-09
15549,중랑구,에스(S) 헤어,서울특별시 중랑구 신내동 479번지 1호 B 중앙하이츠상가-110,2,에스(S) 헤어 미용실,0,검색결과 없음,2020-04-09


In [22]:
naver_re_rejected.to_csv(
      './data/naver_map_seoul_salon_output_rejected_data_2020-04-20_modified.csv'
    , sep=','
    , encoding='utf-8'
)